In [12]:
!pip install pymatgen
!pip install mp_api

In [15]:
from mp_api.client import MPRester
import pandas as pd

# Replace with your actual API key
API_KEY = "iGpMtkXdV09MjR15zn7FzKWfF4uIGpQr"

# Use a with statement to ensure the connection is closed properly
with MPRester(API_KEY) as mpr:
    
    # --- Step 1: Search for promising materials using the .summary endpoint ---
    print("Step 1: Searching for candidate material IDs...")
    summary_docs = mpr.summary.search(
        energy_above_hull=(0, 0.05), # Correct parameter for the summary endpoint
        nelements=(2, 10),
        elements=["O"],
        band_gap=(0.1, None),
        fields=["material_id", "formula_pretty"] # Only request fields available in summary
    )
    
    if not summary_docs:
        print("No materials found matching the criteria in the summary search.")
    else:
        print(f"Found {len(summary_docs)} potential candidates.")
        
        # Extract the material_ids from the summary documents
        material_ids = [doc.material_id for doc in summary_docs]

        # --- Step 2: Retrieve detailed data for the found materials ---
        print("Step 2: Retrieving detailed data for candidate materials...")
        detailed_docs = mpr.materials.search(
            material_ids=material_ids, # Use the list of IDs from Step 1
            fields=[
                "material_id",
                "formula_pretty",
                "structure",
                "band_gap",
                "e_above_hull",
                "density",
                "volume",
            ]
        )
        
        # --- Step 3: Process and save the final detailed data ---
        if detailed_docs:
            print(f"Successfully retrieved detailed data for {len(detailed_docs)} materials.")
            # Convert the list of MaterialDoc objects to a list of dictionaries
            data_for_df = [doc.dict() for doc in detailed_docs]
            
            df = pd.DataFrame(data_for_df)
            df.to_csv("potential_al_cathode_materials.csv", index=False)
            print("Data successfully saved to potential_al_cathode_materials.csv")
        else:
            print("Could not retrieve detailed data for the found material IDs.")

Step 1: Searching for candidate material IDs...


C:\Users\tomas\AppData\Local\Temp\ipykernel_51848\1283186597.py:12: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  summary_docs = mpr.summary.search(
c:\Users\tomas\miniconda3\Lib\site-packages\mp_api\client\routes\materials\summary.py:278: MPRestWarning: You have specified fields used by `_search` that can be understood by `search`
   nelements
To ensure long term support, please use their `search` equivalents:
   num_elements
Please see the documentation:
    `search`: https://materialsproject.github.io/api/_autosummary/mp_api.client.routes.materials.summary.SummaryRester.html#mp_api.client.routes.materials.summary.SummaryRester.search
   `_search`: https://api.materialsproject.org/redoc#tag/Materials-Summary/operation/search_materials_summary__get
  warnings.warn(


Retrieving SummaryDoc documents:   0%|          | 0/28350 [00:00<?, ?it/s]

Found 28350 potential candidates.
Step 2: Retrieving detailed data for candidate materials...


ValueError: List of material/molecule IDs provided is too long. Consider removing the ID filter to automatically pull data for all IDs and filter locally.

In [17]:
from mp_api.client import MPRester
import pandas as pd
import time

# Replace with your actual API key
API_KEY = "iGpMtkXdV09MjR15zn7FzKWfF4uIGpQr"

# Use a with statement to ensure the connection is closed properly
with MPRester(API_KEY) as mpr:
    
    # --- Step 1: Search for promising materials using the .summary endpoint ---
    print("Step 1: Searching for candidate material IDs...")
    # CORRECTED LINE: Changed mpr.summary.search to mpr.materials.summary.search
    summary_docs = mpr.materials.summary.search(
        energy_above_hull=(0, 0.05),
        nelements=(2, 10),
        elements=["O"],
        band_gap=(0.1, None),
        fields=["material_id", "formula_pretty"]
    )
    
    if not summary_docs:
        print("No materials found matching the criteria in the summary search.")
    else:
        print(f"Found {len(summary_docs)} potential candidates.")
        
        material_ids = [doc.material_id for doc in summary_docs]

        # --- Step 2: Retrieve detailed data in chunks ---
        print("Step 2: Retrieving detailed data for candidate materials in batches...")
        
        detailed_docs = []
        chunk_size = 100
        
        for i in range(0, len(material_ids), chunk_size):
            chunk_ids = material_ids[i:i + chunk_size]
            
            print(f"  > Processing chunk {i//chunk_size + 1}/{(len(material_ids) + chunk_size - 1)//chunk_size}...")
            
            # This part is correct and does not need to be changed
            chunk_docs = mpr.materials.search(
                material_ids=chunk_ids,
                fields=[
                    "material_id",
                    "formula_pretty",
                    "structure",
                    "band_gap",
                    "e_above_hull",
                    "density",
                    "volume",
                ]
            )
            detailed_docs.extend(chunk_docs)
            time.sleep(0.1)

        # --- Step 3: Process and save the final detailed data ---
        if detailed_docs:
            print(f"\nSuccessfully retrieved detailed data for {len(detailed_docs)} materials.")
            
            data_for_df = [doc.dict() for doc in detailed_docs]
            
            df = pd.DataFrame(data_for_df)
            df.to_csv("potential_al_cathode_materials.csv", index=False)
            print("Data successfully saved to potential_al_cathode_materials.csv")
        else:
            print("Could not retrieve detailed data for the found material IDs.")

Step 1: Searching for candidate material IDs...


Retrieving SummaryDoc documents:   0%|          | 0/28350 [00:00<?, ?it/s]

Found 28350 potential candidates.
Step 2: Retrieving detailed data for candidate materials in batches...
  > Processing chunk 1/284...


MPRestError: invalid fields requested: ['band_gap', 'e_above_hull']. Available fields: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'material_id', 'structure', 'deprecated', 'deprecation_reasons', 'initial_structures', 'task_ids', 'deprecated_tasks', 'calc_types', 'last_updated', 'created_at', 'origins', 'warnings', 'task_types', 'run_types', 'entries']